In [1]:
# Imports
import pickle
import pandas
import numpy as np
from scipy.stats import binom as scipy_binom
import datetime
from calendar import monthrange
import time as timelib
import matplotlib
matplotlib.use('AGG')
import matplotlib.colors as colors
import matplotlib.pyplot as plt 
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['savefig.bbox'] = 'tight'
matplotlib.rcParams['savefig.pad_inches'] = 0.2 
smallfont = {'family': 'serif', 'size': 12} 
font = {'family': 'serif', 'size': 18} 
bigfont = {'family': 'serif', 'size': 40} 
giantfont = {'family': 'serif', 'size': 80} 
ggiantfont = {'family': 'serif', 'size': 120}
import netCDF4 as nc
import sys 
import os
from os import mkdir
from os.path import join,exists
codedir = "/home/jf4241/ecmwf/tpt_ecmwf"
os.chdir(codedir)
import helper
import strat_feat
import tpt_general

In [ ]:
# Set directories
datadirs = dict({
    "ei": "/scratch/jf4241/ecmwf_data/eraint_data/2022-02-10",
    "e2": "/scratch/jf4241/ecmwf_data/era20c_data/2022-02-10",
    "e5": "/scratch/jf4241/ecmwf_data/era5_data/2022-03-10",
    "s2s": "/scratch/jf4241/ecmwf_data/s2s_data/2021-12-23",
    })
sources = list(datadirs.keys())
featdir = "/scratch/jf4241/ecmwf_data/features/2022-04-25"
if not exists(featdir): mkdir(featdir)
feat_display_dir = join(featdir,"display0")
if not exists(feat_display_dir): mkdir(feat_display_dir)
resultsdir = "/scratch/jf4241/ecmwf_data/results"
if not exists(resultsdir): mkdir(resultsdir)
daydir = join(resultsdir,"2022-04-25")
if not exists(daydir): mkdir(daydir)
expdir = join(daydir,"1")
if not exists(expdir): mkdir(expdir)

In [3]:
!ls

README.md		      hm.out		s2s_request.sbatch
Untitled.ipynb		      output		s2s_request.txt
Untitled1.ipynb		      progress_s2s.txt	s2s_request_october.out
__pycache__		      s2s.err		s2s_request_october.py
driver2.py		      s2s.out		s2s_request_october.txt
flux_dens_tob_tth13624.0.npy  s2s.sbatch	src_dest
flux_dens_tob_tth14368.0.npy  s2s0.out		strat_feat.py
grb2nc.sbatch		      s2s1.out		tdmc_obj.py
grb2nc_era20c.sh	      s2s2.out		tpt_era20c.py
grb2nc_eraint.sh	      s2s_msg.txt	tpt_general.py
grb2nc_s2s.sh		      s2s_request.out	visualize_one_ensemble.py
helper.py		      s2s_request.py


In [4]:
import cartopy

In [5]:
import helper

In [8]:
helper.mantexp(np.pi*1000)

(3.141592653589793, 3)